# This notebook creates a dataset from a session of the IBL Brain Wide Map Dataset

You will need to install the following packages:
- numpy
- pandas
- spikeinterface
- ONE-api
- ibllib

---

In [ ]:
import os
import sys

import numpy as np
import pandas as pd

from one.api import ONE

Install the latest version of SpikeInterface as recommended in the **From source** section [here](https://spikeinterface.readthedocs.io/en/latest/get_started/installation.html).

In [ ]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
from spikeinterface.sortingcomponents.peak_detection import detect_peaks

print(f"SpikeInterface version: {si.__version__}")

Install local functions

In [ ]:
import process_peaks

sys.path.append("..")
import preprocessing
import util

## 1. Read recording session

For this project, we will be using session [sub-CSHL049](https://dandiarchive.org/dandiset/000409/draft/files?location=sub-CSHL049&page=1) of the [IBL Brain Wide Map Dataset](https://dandiarchive.org/dandiset/000409/draft). 

In [ ]:
data_folder = "../data/sub-CSHL049"

os.makedirs(data_folder, exist_ok=True)

In order to obtain this data, we will stream with ONE API using its identifier which is listed in the [metadata](https://api.dandiarchive.org/api/dandisets/000409/versions/draft/assets/7e4fa468-349c-44a9-a482-26898682eed1/).

In [ ]:
one_folder = os.path.join(data_folder, "one")

os.makedirs(one_folder, exist_ok=True)

In [ ]:
one = ONE(
    base_url="https://openalyx.internationalbrainlab.org",
    username="intbrainlab",  
    password="international",
    silent=True
)

eid = "c99d53e6-c317-4c53-99ba-070b26673ac4"
pids, _ = one.eid2pid(eid)
pid = pids[0]

Using SpikeInterface, we can read and save the data to disk. 

In [ ]:
extractors_folder = os.path.join(data_folder, "extractors")

os.makedirs(extractors_folder, exist_ok=True)

In [ ]:
preprocessed_folder = os.path.join(extractors_folder, "preprocessed")

if not os.path.exists(preprocessed_folder): 
    recording = se.read_ibl_recording(eid=eid, stream_name='probe00.ap', cache_folder=one_folder)
    
    # Preprocess the recording
    recording_f = spre.bandpass_filter(recording, freq_min=300, freq_max=6000)
    recording_cmr = spre.common_reference(recording_f, reference='global', operator='median')
    
    # Save the preprocessed recording to disk
    job_kwargs = dict(n_jobs=10, chunk_duration="1s", progress_bar=True)
    recording_cmr.save(folder=preprocessed_folder, **job_kwargs)
else:
    recording_cmr = si.load_extractor(preprocessed_folder)
    
recording_cmr

---

## 2. Create peaks dataset

### Retrieve channel locations

In [ ]:
channel_locations_file = os.path.join(data_folder, "channel_locations.npy")

if not os.path.exists(channel_locations_file):
    channel_locations = preprocessing.extract_channels(recording_cmr)
    np.save(channel_locations_file, channel_locations)
else:
    channel_locations = np.load(channel_locations_file)

display(pd.DataFrame(channel_locations))

### Detect peaks

In [ ]:
peaks_folder = '../data/sub-CSHL049/peaks'
peaks_file = os.path.join(peaks_folder, "peaks.npy")

if os.path.exists(peaks_file):
    peaks_filtered = np.load(peaks_file)
else:
    os.makedirs(peaks_folder, exist_ok=True)
    
    job_kwargs = dict(chunk_duration='1s', n_jobs=10, progress_bar=True)
    
    peaks = detect_peaks(
        recording_cmr,
        method='locally_exclusive',
        peak_sign='neg',
        detect_threshold=6,
        radius_um = 100,
        **job_kwargs
    )    
    
    peaks_filtered = process_peaks.filter_peaks(recording_cmr, peaks)
    
    np.save(peaks_file, peaks_filtered)
    
display(pd.DataFrame(peaks_filtered))

---

## 3. Create dataset

First, create HDF5 files each containing peaks data using `create_peaks_files.sh`. 
You will need to specify the ID of the recording (e.g., 'sub-CSHL049'). This creates multiple HDF5 files based on the number of tasks you set in the jobscript.

Next, run the `combine_peaks_files.py` file to combine the HDF5 files.
You will need to specify the ID of the recording (e.g., 'sub-CSHL049'). This creates a single HDF5 file containing the complete peaks dataset.

Within the file are three datasets:
- channel_locations [384, 3]: A dataset of channel indices and their corresponding locations on the probe.
- properties [n, 3]: A dataset of different properties for each peak - sample_index, channel_index, and amplitude.
- traces [n, 64, 192, 2]: A dataset of traces for each peak.